In [2]:
!pip install transformers

In [3]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [4]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
classifier = pipeline("sentiment-analysis", model = model, tokenizer = tokenizer)
res = classifier(["We are happy to show you the 🤗 Transformers library.",
                  "We hope you don't hate it."])
for result in res:
    print(result)

{'label': 'POSITIVE', 'score': 0.9997753500938416}
{'label': 'NEGATIVE', 'score': 0.5308645963668823}


In [6]:
tokens = tokenizer.tokenize("We are happy to show you the 🤗 Transformers library.")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tokenizer("We are happy to show you the 🤗 Transformers library.")

In [7]:
print(f"Tokens:{tokens}")
print(f"Token IDs: {token_ids}")
print(f"Input IDs: {input_ids}")

Tokens:['we', 'are', 'happy', 'to', 'show', 'you', 'the', '[UNK]', 'transformers', 'library', '.']
Token IDs: [2057, 2024, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012]
Input IDs: {'input_ids': [101, 2057, 2024, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [8]:
X_train = ["We are happy to show you the 🤗 Transformers library.",
                  "We hope you don't hate it."]

batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors="pt") # pt: pytorch
print(batch)

{'input_ids': tensor([[  101,  2057,  2024,  3407,  2000,  2265,  2017,  1996,   100, 19081,
          3075,  1012,   102],
        [  101,  2057,  3246,  2017,  2123,  1005,  1056,  5223,  2009,  1012,
           102,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}


In [9]:
with torch.no_grad():
    outputs = model(**batch, labels=torch.tensor([1,0]))
    print(outputs)
    predictions = F.softmax(outputs.logits, dim=1)
    print(predictions)
    labels = torch.argmax(predictions, dim=1)
    print(labels)
    labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
    print(labels)

SequenceClassifierOutput(loss=tensor(0.3167), logits=tensor([[-4.0742,  4.3266],
        [ 0.0818, -0.0418]]), hidden_states=None, attentions=None)
tensor([[2.2463e-04, 9.9978e-01],
        [5.3086e-01, 4.6914e-01]])
tensor([1, 0])
['POSITIVE', 'NEGATIVE']


In [10]:
save_directory = "saved"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSequenceClassification.from_pretrained(save_directory)

In [11]:
# Sentiment Analysis model for German Language
model_name = "oliverguhr/german-sentiment-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/161 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [18]:
X_train_german = [
    "Mit keinem guten Ergebniss","Das ist gar nicht mal so gut",
    "Total awesome!","nicht so schlecht wie erwartet",
    "Der Test verlief positiv.","Sie fährt ein grünes Auto."]
       
batch = tokenizer(X_train_german, padding=True, truncation=True, max_length=512) # pt: pytorch
batch = torch.tensor(batch['input_ids'])
print(batch)

with torch.no_grad():
    outputs = model(batch)
    print(outputs)
    label_ids = torch.argmax(outputs.logits, dim=1)
    print(label_ids)
    labels = [model.config.id2label[label_id] for label_id in label_ids.tolist()]
    print(labels)

tensor([[    3,   304,  8524,  5569,  2011, 26902,     4,     0,     0],
        [    3,   295,   127,  2523,   149,  2723,   181,  1522,     4],
        [    3, 19990,    18,  7117,  4741, 26982,     4,     0,     0],
        [    3,   149,   181,  6975,   246,  6303,     4,     0,     0],
        [    3,   233,  4496, 14332,  8453, 26914,     4,     0,     0],
        [    3,   371,  9755,    39, 19044, 26902,  3512, 26914,     4]])
SequenceClassifierOutput(loss=None, logits=tensor([[-1.2053,  4.9296, -4.8227],
        [-1.3595,  5.5571, -5.2619],
        [ 4.3217, -0.2520, -5.0253],
        [ 4.0809,  0.1602, -5.3517],
        [-2.2583, -1.4375,  5.2683],
        [-2.4764, -1.4258,  5.5719]]), hidden_states=None, attentions=None)
tensor([1, 1, 0, 0, 2, 2])
['negative', 'negative', 'positive', 'positive', 'neutral', 'neutral']


### For fine tuning the model with custom datasets, visit this link below:
 https://huggingface.co/docs/transformers/training